# C1 W4 Group 8 - Task 3


In [1]:
from PIL import Image

from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH
from tqdm import tqdm

In [2]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [3]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [4]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [5]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [6]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    print(f"File={file.stem}, Aug={augmentation}, Frames={frames}, GT={gt}")

File=00000, Aug=None, Frames=[[np.float64(179.49631381503272), [[64, 41], [2048, 25], [2016, 3653], [196, 3637]]]], GT=[-1]
File=00001, Aug=None-MultiplyHue, Frames=[[np.float64(179.52154949700142), [[19, 38], [509, 37], [496, 665], [17, 661]]]], GT=[150]
File=00002, Aug=None-MultiplyHue, Frames=[[np.float64(0.9495668564358084), [[34, 50], [396, 32], [408, 470], [46, 476]]], [np.float64(0.3015539498682642), [[466, 34], [844, 20], [850, 484], [470, 486]]]], GT=[48, 251]
File=00003, Aug=UnnamedImpulseNoise, Frames=[[np.float64(180.0), [[24, 27], [458, 26], [462, 355], [45, 355]]]], GT=[32]
File=00004, Aug=None-MultiplyHue, Frames=[[np.float64(0.4841781604809512), [[53, 78], [1489, 62], [1497, 2326], [77, 2338]]]], GT=[161]
File=00005, Aug=None, Frames=[[np.float64(179.85346379900508), [[50, 43], [1642, 27], [1646, 2051], [82, 2047]]]], GT=[81]
File=00006, Aug=None, Frames=[[np.float64(1.5481576989779455), [[176, 46], [470, 34], [478, 326], [182, 334]]], [np.float64(1.3510650827196287), [

## Background Removal

In [7]:
from src.background import get_painting_masks, crop_image_by_mask

In [8]:
painting_masks_list_d1 = []
pbar = tqdm(zip(QSD1_W4_PATH_LIST, query_d1_image_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)
    painting_masks_list_d1.append(painting_masks)

Processing 00029: : 30it [00:17,  1.70it/s]


In [9]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(QSD1_W4_PATH_LIST, query_d1_image_PIL_list, painting_masks_list_d1))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = crop_image_by_mask(image, predicted_mask)
    cropped_query_image_list_d1.append(image_list)

Splitting 00029: : 30it [00:01, 29.58it/s]


## Image Denoising

In [10]:
from src.denoising import denoise_image

In [11]:
denoised_query_d1_PIL_list = []
for image_tuple in cropped_query_image_list_d1:
    temp = []
    for img in image_tuple:
        temp.append(denoise_image(img))
    denoised_query_d1_PIL_list.append(temp)

## Retrieval

In [12]:
from src.descriptors import ImageRetrievalSystem, SIFTDescriptor, HOGDescriptor, ORBDescriptor

In [13]:
descriptors = {
    'SIFT': SIFTDescriptor(max_features=500),
    'HOG13': HOGDescriptor(pixels_per_cell=12, cells_per_block=4),
    'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path, log=False)

In [14]:
#Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

descriptor_name = "HOG13"

results[descriptor_name] = retrieval_system.retrieve_similar_images(descriptor_name, database_image_PIL_list, denoised_query_d1_PIL_list, None, K)


Retrieving similar images using HOG13 descriptor:


287it [00:00, 17308.91it/s]
287it [00:00, 26667.37it/s]
287it [00:00, 29414.65it/s]
287it [00:00, 33777.58it/s]
287it [00:00, 24569.65it/s]
287it [00:00, 34953.55it/s]
287it [00:00, 14114.95it/s]
287it [00:00, 34562.15it/s]
287it [00:00, 25898.01it/s]
287it [00:00, 43230.93it/s]
287it [00:00, 27758.91it/s]
287it [00:00, 27219.11it/s]
287it [00:00, 22420.24it/s]
287it [00:00, 27047.26it/s]
287it [00:00, 39476.77it/s]
287it [00:00, 34076.86it/s]
287it [00:00, 26126.78it/s]
287it [00:00, 27937.37it/s]
287it [00:00, 38105.90it/s]
287it [00:00, 37550.78it/s]
287it [00:00, 39821.54it/s]
287it [00:00, 28858.28it/s]
287it [00:00, 17900.39it/s]
287it [00:00, 32273.39it/s]
287it [00:00, 39861.10it/s]
287it [00:00, 41629.73it/s]
287it [00:00, 40056.08it/s]
287it [00:00, 39111.22it/s]
287it [00:00, 41793.05it/s]
287it [00:00, 39256.63it/s]
287it [00:00, 37857.82it/s]
287it [00:00, 40773.81it/s]
287it [00:00, 39972.28it/s]
287it [00:00, 40388.03it/s]
287it [00:00, 40085.42it/s]
287it [00:00, 27782.

## Evaluation

In [15]:
from src.metrics import MeanAveragePrecisionAtK

map = MeanAveragePrecisionAtK()

In [16]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.append([element[0]])
        GT_mod.append([element[1]])
    else:
        GT_mod.append([element[0]])

retrieved_images_d1 = results[descriptor_name]
results_mod = []
for element in retrieved_images_d1:
    for e in element:
        results_mod.append(e)

print(GT_mod)
print(results_mod)

[[-1], [150], [48], [251], [32], [161], [81], [62], [38], [-1], [128], [155], [258], [136], [76], [-1], [-1], [53], [-1], [12], [11], [280], [-1], [182], [252], [-1], [272], [117], [-1], [242], [260], [94], [132], [223], [-1], [127], [47], [13], [-1]]
[[-1], [-1], [48, 21, 11, 92, 35, 46, 238, 93, 139, 40], [-1], [32, 11, 104, 282, 40, 140, 193, 238, 21, 88], [161, 11, 40, 193, 238, 88, 104, 93, 254, 92], [81, 11, 40, 93, 54, 238, 88, 104, 32, 92], [-1], [-1], [-1], [-1], [-1], [76, 191, 40, 21, 253, 16, 74, 249, 43, 46], [-1], [11, 40, 54, 238, 32, 195, 282, 21, 104, 60], [53, 238, 54, 254, 11, 92, 93, 21, 74, 46], [93, 197, 46, 198, 78, 274, 11, 21, 250, 156], [12, 92, 21, 93, 74, 11, 78, 107, 156, 25], [11, 238, 54, 282, 40, 104, 174, 246, 92, 21], [-1], [169, 26, 37, 5, 220, 165, 66, 108, 240, 61], [-1], [233, 21, 191, 57, 74, 3, 282, 92, 40, 11], [252, 215, 237, 133, 29, 58, 57, 218, 259, 163], [93, 78, 46, 197, 156, 16, 198, 274, 92, 21], [176, 210, 214, 255, 40, 191, 282, 194, 2

In [17]:
result = map.compute(GT_mod, results_mod, k=10)
print(f"MAP QSD1_W4 Method Week 3: {result}")

MAP QSD1_W4 Method Week 3: 0.19444444444444445
